In [8]:
pip install tensorflow numpy matplotlib


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = 'C:/Users/Stefania/Desktop/ACS/VAF/images' #calea catre fisierul cu imagini

train_dir = f'{base_dir}/train'
validation_dir = f'{base_dir}/val'

# Crearea generatorului de date de antrenament cu augmentare
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Crearea generatorului de date de validare (fara augmentare, doar scalare)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Configurarea generatorului de imagini de antrenament
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Redimensionarea imaginilor la 150x150
    batch_size=20,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

# Configurarea generatorului de imagini de validare
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Initializarea modelului
model = Sequential()

# Adaugarea primului strat convolutional si a unui strat de pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

# Adaugarea unui al doilea strat convolutional si de pooling
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

# Adaugarea unui al treilea strat convolutional si de pooling
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

# Adaugarea unui al patrulea strat convolutional si de pooling
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

# Flattening rezultatelor pentru a le trece prin straturi dense (fully connected)
model.add(Flatten())

# Adaugarea unui strat dens
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

# Adaugarea unui strat dens suplimentar
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Stratul de iesire
model.add(Dense(1, activation='sigmoid'))

# Compilarea modelului
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [12]:
# Antrenarea modelului
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Numarul de loturi care vor fi rulate pentru fiecare epoca
    epochs=20,  # Numarul de epoci de antrenament
    validation_data=validation_generator,
    validation_steps=50  # Numarul de loturi de validare pentru a rula la sfarsitul fiecarei epoci
)


Epoch 1/20
100/100 [==============================] - 57s 535ms/step - loss: 1.8167 - accuracy: 0.7640 - val_loss: 36.3957 - val_accuracy: 0.5000
Epoch 2/20
100/100 [==============================] - 51s 507ms/step - loss: 1.0548 - accuracy: 0.7900
Epoch 3/20
100/100 [==============================] - 50s 500ms/step - loss: 0.7924 - accuracy: 0.8130
Epoch 4/20
100/100 [==============================] - 51s 506ms/step - loss: 0.5000 - accuracy: 0.8292
Epoch 5/20
100/100 [==============================] - 50s 493ms/step - loss: 0.3926 - accuracy: 0.8200
Epoch 6/20
100/100 [==============================] - 51s 506ms/step - loss: 0.3519 - accuracy: 0.8475
Epoch 7/20
100/100 [==============================] - 50s 496ms/step - loss: 0.3662 - accuracy: 0.8400
Epoch 8/20
100/100 [==============================] - 50s 496ms/step - loss: 0.3925 - accuracy: 0.8317
Epoch 9/20
100/100 [==============================] - 49s 487ms/step - loss: 0.3042 - accuracy: 0.8790
Epoch 10/20
100/100 [=========

In [32]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

def preprocess_image(image_path, target_size=(150, 150)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return img_array_expanded_dims / 255.0

def process_folder(folder_path, true_label):
    images = os.listdir(folder_path)
    correct_predictions = 0
    total_images = 0
    
    for image in images:
        image_path = os.path.join(folder_path, image)
        try:
            preprocessed_image = preprocess_image(image_path)
            prediction = model.predict(preprocessed_image)
            predicted_label = "PNEUMONIA" if prediction[0][0] > 0.5 else "NORMAL"
            if predicted_label == true_label:
                correct_predictions += 1
            total_images += 1
            print(f"{image}: {predicted_label}")
        except Exception as e:
            print(f"Eroare la procesarea imaginii {image}: {e}")
    
    accuracy = correct_predictions / total_images if total_images > 0 else 0
    print(f"Exactitate pentru {true_label}: {accuracy:.2f} ({correct_predictions}/{total_images})")
    return correct_predictions, total_images, accuracy

# Calea către folderele cu imagini
folder_pneumonia = 'C:/Users/Stefania/Desktop/ACS/VAF/images/test/PNEUMONIA'
folder_normal = 'C:/Users/Stefania/Desktop/ACS/VAF/images/test/NORMAL'

# Procesează fiecare folder și calculează exactitatea
print("Procesarea imaginilor cu PNEUMONIA:")
correct_pneumonia, total_pneumonia, accuracy_pneumonia = process_folder(folder_pneumonia, "PNEUMONIA")

print("Procesarea imaginilor NORMALE:")
correct_normal, total_normal, accuracy_normal = process_folder(folder_normal, "NORMAL")


Procesarea imaginilor cu PNEUMONIA:
1/1 [==============================] - 0s 37ms/step
person100_bacteria_475.jpeg: PNEUMONIA
1/1 [==============================] - 0s 38ms/step
person100_bacteria_477.jpeg: PNEUMONIA
1/1 [==============================] - 0s 37ms/step
person100_bacteria_478.jpeg: PNEUMONIA
1/1 [==============================] - 0s 36ms/step
person100_bacteria_479.jpeg: PNEUMONIA
1/1 [==============================] - 0s 35ms/step
person100_bacteria_480.jpeg: PNEUMONIA
1/1 [==============================] - 0s 36ms/step
person100_bacteria_481.jpeg: PNEUMONIA
1/1 [==============================] - 0s 35ms/step
person100_bacteria_482.jpeg: PNEUMONIA
1/1 [==============================] - 0s 39ms/step
person101_bacteria_483.jpeg: PNEUMONIA
1/1 [==============================] - 0s 35ms/step
person101_bacteria_484.jpeg: NORMAL
1/1 [==============================] - 0s 36ms/step
person101_bacteria_485.jpeg: PNEUMONIA
1/1 [==============================] - 0s 37ms/step
perso

1/1 [==============================] - 0s 28ms/step
person161_bacteria_757.jpeg: PNEUMONIA
1/1 [==============================] - 0s 27ms/step
person161_bacteria_759.jpeg: PNEUMONIA
1/1 [==============================] - 0s 27ms/step
person161_bacteria_762.jpeg: PNEUMONIA
1/1 [==============================] - 0s 26ms/step
person1620_virus_2807.jpeg: NORMAL
1/1 [==============================] - 0s 28ms/step
person1622_virus_2810.jpeg: PNEUMONIA
1/1 [==============================] - 0s 29ms/step
person1623_virus_2813.jpeg: NORMAL
1/1 [==============================] - 0s 29ms/step
person1625_virus_2817.jpeg: NORMAL
1/1 [==============================] - 0s 28ms/step
person1626_virus_2818.jpeg: PNEUMONIA
1/1 [==============================] - 0s 28ms/step
person1627_virus_2819.jpeg: PNEUMONIA
1/1 [==============================] - 0s 28ms/step
person1628_virus_2821.jpeg: NORMAL
1/1 [==============================] - 0s 27ms/step
person1628_virus_2822.jpeg: PNEUMONIA
1/1 [==============

1/1 [==============================] - 0s 29ms/step
person8_virus_28.jpeg: PNEUMONIA
1/1 [==============================] - 0s 29ms/step
person90_bacteria_442.jpeg: PNEUMONIA
1/1 [==============================] - 0s 28ms/step
person90_bacteria_443.jpeg: PNEUMONIA
1/1 [==============================] - 0s 30ms/step
person91_bacteria_445.jpeg: PNEUMONIA
1/1 [==============================] - 0s 28ms/step
person91_bacteria_446.jpeg: PNEUMONIA
1/1 [==============================] - 0s 28ms/step
person91_bacteria_447.jpeg: PNEUMONIA
1/1 [==============================] - 0s 30ms/step
person91_bacteria_448.jpeg: PNEUMONIA
1/1 [==============================] - 0s 29ms/step
person91_bacteria_449.jpeg: PNEUMONIA
1/1 [==============================] - 0s 30ms/step
person92_bacteria_450.jpeg: PNEUMONIA
1/1 [==============================] - 0s 29ms/step
person92_bacteria_451.jpeg: PNEUMONIA
1/1 [==============================] - 0s 27ms/step
person93_bacteria_453.jpeg: NORMAL
1/1 [=============

NORMAL2-IM-0030-0001.jpeg: NORMAL
1/1 [==============================] - 0s 29ms/step
NORMAL2-IM-0033-0001.jpeg: NORMAL
1/1 [==============================] - 0s 28ms/step
NORMAL2-IM-0035-0001.jpeg: NORMAL
1/1 [==============================] - 0s 27ms/step
NORMAL2-IM-0041-0001.jpeg: NORMAL
1/1 [==============================] - 0s 27ms/step
NORMAL2-IM-0045-0001.jpeg: NORMAL
1/1 [==============================] - 0s 28ms/step
NORMAL2-IM-0051-0001.jpeg: NORMAL
1/1 [==============================] - 0s 31ms/step
NORMAL2-IM-0052-0001.jpeg: PNEUMONIA
1/1 [==============================] - 0s 28ms/step
NORMAL2-IM-0058-0001.jpeg: NORMAL
1/1 [==============================] - 0s 29ms/step
NORMAL2-IM-0059-0001.jpeg: NORMAL
1/1 [==============================] - 0s 28ms/step
NORMAL2-IM-0060-0001.jpeg: NORMAL
1/1 [==============================] - 0s 28ms/step
NORMAL2-IM-0066-0001.jpeg: NORMAL
1/1 [==============================] - 0s 29ms/step
NORMAL2-IM-0072-0001.jpeg: NORMAL
1/1 [============